TorchText có các hàm giúp chúng ta xây dựng Dataset từ dữ liệu bên ngoài 


In [1]:
# ! pip install torchtext==0.9.1
# Nên dùng bản 0.9 

# Xử lý data ở dạng json 

In [2]:
import torch
from torchtext.legacy import data, datasets

In [3]:
NAME = data.Field()
PLACE = data.Field()
SAYING = data.Field()

Tiếp theo ta cần thông báo cho torchtext trường hợp nào áp dụng cho `json` nào 

Ta truy cập trường `TEXT` và `LABEL` trong quá trình train/evaluation bằng cách sử dụng `batch.text` và `batch.label`. Ta truy cập được vào do torchtext đã có sẵn 2 attribute `TEXT` và `LABEL` trong class, mỗi `attrabute` là một tensor chứa text và label 


Một sóo lưu ý quan trọng: 
 - Thứ tự của Key trong dictionary `fields` không quan trọng, nhưng nó phải match với keys trong json data
 - Tên của Fields khônng nhất thiết phải match với keys trong json object Ví dụ ta có thể sử dụng PLACE thay cho trường location 
 - Nếu values của json là string, Fields tokeniztion sẽ được áp dụng lên nó (mặc định sẽ là ngăn cách bởi spaces). Tuy nhiên, nếu values là một list, tokenization sẽ không được áp dụng lên nó 
 ta có thể sử dụng `Field` để tokenize data. Ví dụ: 
  ```python
    TEXT = Field(tokenize=get_tokenizer("basic_english"))
    ```
    - Nếu ta muốn sử dụng `Field` để tokenize data, ta cần phải sử dụng `Field` cho cả text và label. Ví dụ:
    ```python
    TEXT = Field(tokenize=get_tokenizer("basic_english"))
    LABEL = Field(sequential=False)
    fields = {"text": ("text", TEXT), "label": ("label", LABEL)}
    ```
- Thông thường, ta nên tokenize data trước giúp tiết kiệm thời gian để torchtext không cần tokenize nữa 
- Value trong các trường của json không nhất thiết phải cùng 1 kiểu 
- Nếu sử dụng trường trong json, tất cả các single example phải là instance của trường đó. Trong ví dụ dưới đây, tất cả các example phải có NAME, LOCATION, QUOTE. Tuy nhiên, không ta không dùng trường AGE nên examples thiếu trường AGE cũng không ảnh hưởng gì 



In [4]:
fields = {'name': ("n", NAME), 'location': ('p', PLACE), 'quote': ('s', SAYING)} 

Bây giờ trong training loop ta có thể lặp iterator và truy cập tên thông qua batch.n, location thông qua batch.p, và quote thông qua batch.s

In [20]:
train_data, test_data  = data.TabularDataset.splits(path = '../data', # chọn nơi lưu data
                                                    train = 'train.json',       # tên file trong thư mục lưu cho train 
                                                    test = 'test.json',        # tên file trong thư mục lưu cho test
                                                    format = 'json', # định dạng file
                                                    fields = fields # các trường dữ liệu
                                                    ) 



lưu ý: ở đây các tên trường (field names)  là n, p, s khớp với những gì đc define trong `fields` dictionary 


bên cạnh đó cần chú ý, từ "United Kingdom" trong p được tokenize trong khi "united kingdom" trong s không được tokenize. lý do là TorchText sẽ không áp dụng tokenize khi value có kiểu dữ liệu là list 

In [19]:
vars(train_data[0]) 

{'n': ['John'],
 'p': ['United', 'Kingdom'],
 's': ['i', 'love', 'the', 'united kingdom']}

Từ bây giờ, ta có thể sử dụng train_data, test_data, và valid_data để xây dựng vocabulary và tạo iterators. Ta có thể sử dụng các attribute  bằng cách sử dụng batch.n, batch.p và  batch.s để lấy name.s places và sayings 

In [23]:
train_data 

Reading CSV/TSV 